##Redshift Data Load


In [1]:
#pip install --upgrade boto3
#!pip install awswrangler
#!pip list | grep boto3

In [10]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

redshift = boto3.client('redshift')
secretsmanager = boto3.client('secretsmanager')

In [11]:
import json

secret = secretsmanager.get_secret_value(SecretId='team3_redshift_login')
cred = json.loads(secret['SecretString'])

master_user_name = cred[0]['username']
master_user_pw = cred[1]['password']

In [13]:
redshift_cluster_identifier = 'team3'

database_name_redshift = 'dev'

redshift_port = '5439'

schema_redshift = 'redshift'

print(master_user_name)

team3


In [14]:
import time

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

available


In [16]:
redshift_endpoint_address = response['Clusters'][0]['Endpoint']['Address']
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

print('Redshift endpoint: {}'.format(redshift_endpoint_address))
print('IAM Role: {}'.format(iam_role))

Redshift endpoint: team3.cgkvnybhfypi.us-east-1.redshift.amazonaws.com
IAM Role: arn:aws:iam::211125778552:role/team3-Redshift-role


In [17]:
print(response['Clusters'])

[{'ClusterIdentifier': 'team3', 'NodeType': 'dc2.large', 'ClusterStatus': 'available', 'ClusterAvailabilityStatus': 'Available', 'MasterUsername': 'team3', 'DBName': 'dev', 'Endpoint': {'Address': 'team3.cgkvnybhfypi.us-east-1.redshift.amazonaws.com', 'Port': 5439}, 'ClusterCreateTime': datetime.datetime(2024, 4, 7, 20, 43, 19, 265000, tzinfo=tzlocal()), 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-07248e327a8dc9548', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'default', 'VpcId': 'vpc-0a19e74ac58edb30f', 'AvailabilityZone': 'us-east-1a', 'PreferredMaintenanceWindow': 'mon:03:00-mon:03:30', 'PendingModifiedValues': {}, 'ClusterVersion': '1.0', 'AllowVersionUpgrade': True, 'NumberOfNodes': 2, 'PubliclyAccessible': False, 'Encrypted': False, 'ClusterPublicKey': 'ssh-r

In [18]:
import awswrangler as wr

con_redshift = wr.data_api.redshift.connect(
    cluster_id=redshift_cluster_identifier,
    database=database_name_redshift,
    db_user=master_user_name,
)

schema_redshift = 'team3'

statement = """CREATE SCHEMA IF NOT EXISTS {}""".format(schema_redshift)

wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

""


In [ ]:
Redhisft_role ='team3-Redshift-role'

In [26]:
create_table_query="""

CREATE TABLE IF NOT EXISTS team3.output_csv_gzip (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_csv_lz (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_csv_snappy (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_json_gzip (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_json_lz (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_json_snappy  (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_orc_snappy (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );
CREATE TABLE IF NOT EXISTS team3.output_orc_zlib (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );
CREATE TABLE IF NOT EXISTS team3.output_parquet_gzip (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_parquet_lz4 (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_parquet_lzo (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );
CREATE TABLE IF NOT EXISTS team3.output_parquet_snappy (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );

CREATE TABLE IF NOT EXISTS team3.output_parquet_zstd (
                finaldisposition varchar(256), sentencetime bigint, court varchar(256), complainant varchar(256), publicdefender boolean, gender varchar(256), race varchar(256), casetype varchar(256), class varchar(256), codesection varchar(256), chargeamended boolean
            );
""" 

In [28]:
import boto3

# Establish connections
glue = boto3.client('glue')
redshift = boto3.client('redshift-data')

# Define Glue database and Redshift schema
glue_database = 'team3-court-data'
redshift_schema = schema_redshift

# Get table list from Glue Catalog
tables = glue.get_tables(DatabaseName=glue_database)['TableList']

# Execute CREATE TABLE query
try:
    redshift.execute_statement(
        ClusterIdentifier='YOUR_REDSHIFT_CLUSTER_ID',
        Database='YOUR_REDSHIFT_DATABASE_NAME',
        DbUser='YOUR_REDSHIFT_USER',
        Sql=create_table_query
        )
    print(f"Table {table_name} created successfully.")
except Exception as e:
    print(f"Error creating table {table_name}: {e}")


Error creating table output_parquet_zstd: An error occurred (ValidationException) when calling the ExecuteStatement operation: Redshift endpoint doesn't exist in this region.


In [ ]:
import boto3

# Establish connections
glue = boto3.client('glue')
redshift = boto3.client('redshift-data')

# Define Glue database and Redshift schema
glue_database = 'team3-court-data'
redshift_schema = schema_redshift

# Get table list from Glue Catalog
tables = glue.get_tables(DatabaseName=glue_database)['TableList']

# Iterate through tables and load data into Redshift
for table in tables:
    table_name = table['Name']
    table_location = table.get('StorageDescriptor', {}).get('Location')

    if table_location:
        # Construct column definitions
        column_definitions = []
        columns = table.get('StorageDescriptor', {}).get('Columns', [])
        for col in columns:
            col_name = col['Name']
            col_type = col['Type']
            column_definitions.append(f"{col_name} {col_type}")

        # Construct CREATE TABLE query
        create_table_query = f"""
            CREATE TABLE IF NOT EXISTS {redshift_schema}.{table_name} (
                {', '.join(column_definitions)}
            );
        """

        print(create_table_query)
    
        # Construct COPY query with error handling and logging
        copy_query = f"""
            COPY {redshift_schema}.{table_name}
            FROM 's3://{table_location}'
            iam_role team3-Redshift-role
            FORMAT AS PARQUET
            IGNOREHEADER 1
            EMPTYASNULL
            BLANKSASNULL;
        """

        print(copy_query)

        # Execute CREATE TABLE query
        try:
            redshift.execute_statement(
                ClusterIdentifier='YOUR_REDSHIFT_CLUSTER_ID',
                Database='YOUR_REDSHIFT_DATABASE_NAME',
                DbUser='YOUR_REDSHIFT_USER',
                Sql=create_table_query
            )
            print(f"Table {table_name} created successfully.")
        except Exception as e:
            print(f"Error creating table {table_name}: {e}")

        try:
            redshift.execute_statement(
                ClusterIdentifier='team3',
                Database='court-data',
                DbUser='team3',
                Sql=copy_query
            )
            print(f"Table {table_name} loaded successfully.")
        except Exception as e:
            print(f"Error loading table {table_name}: {e}")
    else:
        print(f"Table {table_name} does not have a StorageDescriptor with Location. Skipping...")



            CREATE TABLE IF NOT EXISTS team3.model_data_csv (
                finaldisposition string, sentencetime bigint, court string, complainant string, publicdefender boolean, gender string, race string, casetype string, class string, codesection string, chargeamended boolean
            );
        

            COPY team3.model_data_csv
            FROM 's3://s3://team-3-project-data/convrted-data/redshift-ingestion/model_data.csv'
            iam_role 'team3-Redshift-role'
            FORMAT AS PARQUET
            IGNOREHEADER 1
            CSV QUOTE AS '"'
            EMPTYASNULL
            BLANKSASNULL;
        
Error creating table model_data_csv: An error occurred (ValidationException) when calling the ExecuteStatement operation: Redshift endpoint doesn't exist in this region.
Table model_data_csv loaded successfully.

            CREATE TABLE IF NOT EXISTS team3.model_data_csv_gz (
                finaldisposition string, sentencetime bigint, court string, complainant st

In [20]:
print(glue.get_table(DatabaseName=glue_database, Name='model_data_csv')['Table']['StorageDescriptor'])

{'Columns': [{'Name': 'finaldisposition', 'Type': 'string'}, {'Name': 'sentencetime', 'Type': 'bigint'}, {'Name': 'court', 'Type': 'string'}, {'Name': 'complainant', 'Type': 'string'}, {'Name': 'publicdefender', 'Type': 'boolean'}, {'Name': 'gender', 'Type': 'string'}, {'Name': 'race', 'Type': 'string'}, {'Name': 'casetype', 'Type': 'string'}, {'Name': 'class', 'Type': 'string'}, {'Name': 'codesection', 'Type': 'string'}, {'Name': 'chargeamended', 'Type': 'boolean'}], 'Location': 's3://team-3-project-data/convrted-data/redshift-ingestion/model_data.csv', 'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat', 'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat', 'Compressed': False, 'NumberOfBuckets': -1, 'SerdeInfo': {'SerializationLibrary': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe', 'Parameters': {'field.delim': ','}}, 'BucketColumns': [], 'SortColumns': [], 'Parameters': {'skip.header.line.count': '1', 'sizeKey': '807368614', 'UPDATED_BY_CRAWLER

In [19]:
import time

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

available


In [20]:
redshift_endpoint_address = response['Clusters'][0]['Endpoint']['Address']
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

print('Redshift endpoint: {}'.format(redshift_endpoint_address))
print('IAM Role: {}'.format(iam_role))

Redshift endpoint: team3.cgkvnybhfypi.us-east-1.redshift.amazonaws.com
IAM Role: arn:aws:iam::211125778552:role/team3-Redshift-role


In [21]:
import awswrangler as wr

con_redshift = wr.data_api.redshift.connect(
    cluster_id=redshift_cluster_identifier,
    database=database_name_redshift,
    db_user=master_user_name,
)

#Athena Database 


In [23]:
schema_athena = 'team3-court-data'
database_name_athena = 'team3-court-data'
#arn:aws:iam::211125778552:role/team3-Redshift-role
iam_role ='arn:aws:iam::211125778552:role/team3-Redshift-role' 
region_name='us-east-1'

In [24]:
statement = """
CREATE EXTERNAL SCHEMA IF NOT EXISTS {} FROM DATA CATALOG 
    DATABASE '{}' 
    IAM_ROLE '{}'
    REGION '{}'
    CREATE EXTERNAL DATABASE IF NOT EXISTS
""".format(schema_athena, database_name_athena, iam_role, region_name)

print(statement)


CREATE EXTERNAL SCHEMA IF NOT EXISTS team3-court-data FROM DATA CATALOG 
    DATABASE 'team3-court-data' 
    IAM_ROLE 'team3-Redshift-role'
    REGION 'us-east-1'
    CREATE EXTERNAL DATABASE IF NOT EXISTS



In [25]:
wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

RedshiftDataApiFailedException: Request 3a9e276c-5a4d-4157-93a4-3c7446260395 failed with status FAILED and error ERROR: syntax error at or near "-"
  Position: 44

In [ ]:
wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)